In [1]:
! nvidia-smi

Thu Oct 20 16:17:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.48.07    Driver Version: 515.48.07    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:03:00.0 Off |                    0 |
| N/A   27C    P0    50W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:41:00.0 Off |                    0 |
| N/A   

In [ ]:
! nproc

In [ ]:
! nram

In [ ]:
pip install torch

In [ ]:
pip install albumentations

In [ ]:
pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
pip install torchvision.models.segmentation.deeplabv3 

In [ ]:
! free

In [1]:
class Material:
 
  def __init__(self, name, input_rgb_vals, output_val, confidence_threshold=0):
    self.name = name
    self.input_rgb_vals = input_rgb_vals
    self.output_val = output_val
    self.confidence_threshold = confidence_threshold
 
#Creating a list of materials so we can iterate through it
materials = [
             Material("amf", [50,50,50], 50, 0.55),
             Material("background", [255,255,255], 255, 0.4),
             
             ]
 
num_materials =len(materials)
#Various input/output directories
training_image_directory = "/pscratch/sd/d/drippner/Reslice Images/"
training_mask_directory = "/pscratch/sd/d/drippner/Reslice Masks/"
#Fraction of total annotations you want to leave for validating the model.
validation_fraction=0.2
#Model Performance varies, make multiple models to have the best chance at success.
num_models=5
#Model Performance improves with increasing epochs, to a point.
num_epochs=500
"""Increasing batch size increase model training speed, but also eats up VRAM on the GPU. Find a balance between scale and batch size
that best suits your needs"""
batch_size=2
#Decrease scale to decrease VRAM usage; if you run out of VRAM during traing, restart your runtime and down scale your images
scale=1
#Input model directory
models_directory = "/pscratch/sd/d/drippner/best_models/"
#Input the name you want to use for your group of models
model_group='new amf p2 1p 100 epoch/'
current_model_name = 'new amf p2 1p 100 epoch'
"""Hold images/annotations in reserve to test your model performance. Use this metric to decide which model you want to use 
for your data analysis"""
test_images = "/pscratch/sd/d/drippner/Reslice Images/"
test_masks= "/pscratch/sd/d/drippner/Reslice Masks/"
csv_directory = "/pscratch/sd/d/drippner/amf test deeplabv3.csv"
#Input the directory of the data you want to segment here.
# inference_directory= 'drive/MyDrive/dani/column_jpegs/Column1JPEG/'
#inference_directory= 'drive/MyDrive/ALS Workflow/tiina new soil/'
# inference_directory= 'drive/MyDrive/ALS Workflow/unf soil aggregate/rec20200215_203743_Devin_UNFsoilaggregate_convertedint/'
inference_directory="/pscratch/sd/d/drippner/Reslice Images/"

#Input the 5 alpha-numeric characters proceding the file number of your images
  #EX. Jmic3111_S0_GRID image_0.tif ----->mage_
proceeding="1h1"
#Input the 4 or mor alpha-numeric characters following the file number
  #EX. Jmic3111_S0_GRID image_0.tif ----->.tif
following=".png"
output_directory = "/pscratch/sd/d/drippner/amf test/"

In [2]:
#Code Box 4
from os.path import splitext
from os import listdir
import numpy as np
from glob import glob
import torch
from torch.utils.data import Dataset
import logging
from PIL import Image
import random
#import scipy.ndimage as ndi
import albumentations as A
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from scipy.ndimage import morphology
from torch.utils.data import DataLoader, random_split

import pandas as pd
import re

 
class BasicDataset(Dataset):
    def __init__(self, imgs_dir, masks_dir, scale=scale, transform=False):
        self.imgs_dir = imgs_dir
        self.masks_dir = masks_dir
        self.scale = scale
        self.transform=transform
        self.t_list=A.Compose([A.HorizontalFlip(p=0.4),A.VerticalFlip(p=0.4), A.Rotate(limit=(-50, 50), p=0.4),])
        self.means=[0,0,0]
        self.stds=[1,1,1]

        
        assert 0 < scale <= 1, 'Scale must be between 0 and 1'
 
        self.ids = [splitext(file)[0] for file in listdir(imgs_dir)
                    if not file.startswith('.')]
        logging.info(f'Creating dataset with {len(self.ids)} examples')
 
    def __len__(self):
        return len(self.ids)
 
 
    @classmethod
    def mask_preprocess(cls, pil_img, scale):
        w, h = pil_img.size
        newW, newH = int(scale * w), int(scale * h)
        assert newW > 0 and newH > 0, 'Scale is too small'
        pil_img = pil_img.resize((newW, newH))
 
        img_nd = np.array(pil_img)
 
        if len(img_nd.shape) == 2:
            img_nd = np.expand_dims(img_nd, axis=2)
 
       
        return img_nd
    
 
        
    def img_preprocess(cls, pil_img, scale):
        w, h = pil_img.size
        newW, newH = int(scale * w), int(scale * h)
        assert newW > 0 and newH > 0, 'Scale is too small'
        pil_img = pil_img.resize((newW, newH))
 
        img_nd = np.array(pil_img)
 
        if len(img_nd.shape) == 2:
            img_nd = np.expand_dims(img_nd, axis=2)
 
       
 
        return img_nd
 
    def __getitem__(self, i):
        idx = self.ids[i]
        mask_file = glob(self.masks_dir + idx + '*')
        img_file = glob(self.imgs_dir + idx + '*')
 
        assert len(mask_file) == 1, \
            f'Either no mask or multiple masks found for the ID {idx}: {mask_file}'
        assert len(img_file) == 1, \
            f'Either no image or multiple images found for the ID {idx}: {img_file}'
        mask = Image.open(mask_file[0])
        img = Image.open(img_file[0])
 
  
        
 
        
        #Reshapes from 1 channel to 3 channels in grayscale
        img = self.img_preprocess(img, self.scale)
        mask = self.mask_preprocess(mask, self.scale)
        new_image=np.zeros((img.shape[0],img.shape[1],3))
        new_image[:,:,0]=img[:,:,0]
        new_image[:,:,1]=img[:,:,0]
        new_image[:,:,2]=img[:,:,0]
        
 
 
 
        img=new_image
 
        #New Code
        masklist=[]
            
        for i, mat in enumerate(materials):
        
          indices = np.all(mask == mat.input_rgb_vals, axis=-1)
          new_mask=np.zeros((img.shape[0],img.shape[1]))
          new_mask[indices] = 1
          masklist.append(new_mask)
 
        mask=masklist
  
 
        
        if img.max() > 1:
            img = img / 255
 
       
 
        
        if self.transform:
            augmented=self.t_list(image=img, masks=mask)
            img=augmented["image"]
            mask=augmented["masks"]
            
 
        
 
        img = img.transpose((2, 0, 1))
        
        mask=np.array(mask)
        
        
 
        
 
        img=torch.from_numpy(img)
        mask=torch.from_numpy(mask)
        
        img=transforms.Normalize(mean=self.means, std=self.stds)(img)
        return img, mask #img_file[0]

class InferenceDataset(Dataset):
    def __init__(self, imgs_dir, scale=scale, transform=False):
        self.imgs_dir = imgs_dir
        # self.masks_dir = masks_dir
        self.scale = scale
        self.transform=transform
        self.t_list=A.Compose([A.HorizontalFlip(p=0.4),A.VerticalFlip(p=0.4), A.Rotate(limit=(-50, 50), p=0.4),])
        self.means=[0,0,0]
        self.stds=[1,1,1]

        
        assert 0 < scale <= 1, 'Scale must be between 0 and 1'
 
        self.ids = [splitext(file)[0] for file in listdir(imgs_dir)
                    if not file.startswith('.')]
        logging.info(f'Creating dataset with {len(self.ids)} examples')
 
    def __len__(self):
        return len(self.ids)
 
 
    @classmethod
    def mask_preprocess(cls, pil_img, scale):
        w, h = pil_img.size
        newW, newH = int(scale * w), int(scale * h)
        assert newW > 0 and newH > 0, 'Scale is too small'
        pil_img = pil_img.resize((newW, newH))
 
        img_nd = np.array(pil_img)
 
        if len(img_nd.shape) == 2:
            img_nd = np.expand_dims(img_nd, axis=2)
 
       
        return img_nd
    
 
        
    def img_preprocess(cls, pil_img, scale):
        w, h = pil_img.size
        newW, newH = int(scale * w), int(scale * h)
        assert newW > 0 and newH > 0, 'Scale is too small'
        pil_img = pil_img.resize((newW, newH))
 
        img_nd = np.array(pil_img)
 
        if len(img_nd.shape) == 2:
            img_nd = np.expand_dims(img_nd, axis=2)
 
       
 
        return img_nd
 
    def __getitem__(self, i):
        idx = self.ids[i]
        # mask_file = glob(self.masks_dir + idx + '*')
        img_file = glob(self.imgs_dir + idx + '*')
 
        # assert len(mask_file) == 1, \
        #     f'Either no mask or multiple masks found for the ID {idx}: {mask_file}'
        assert len(img_file) == 1, \
            f'Either no image or multiple images found for the ID {idx}: {img_file}'
        # mask = Image.open(mask_file[0])
        img = Image.open(img_file[0])
 
  
        
 
        
        #Reshapes from 1 channel to 3 channels in grayscale
        img = self.img_preprocess(img, self.scale)
        # mask = self.mask_preprocess(mask, self.scale)
        new_image=np.zeros((img.shape[0],img.shape[1],3))
        new_image[:,:,0]=img[:,:,0]
        new_image[:,:,1]=img[:,:,0]
        new_image[:,:,2]=img[:,:,0]
        
 
 
 
        img=new_image
 
        #New Code
        # masklist=[]
            
        # for i, mat in enumerate(materials):
        
        #   indices = np.all(mask == mat.input_rgb_vals, axis=-1)
        #   new_mask=np.zeros((img.shape[0],img.shape[1]))
        #   new_mask[indices] = 1
        #   masklist.append(new_mask)
 
        # mask=masklist
  
 
        
        if img.max() > 1:
            img = img / 255
 
       
 
        
        if self.transform:
            augmented=self.t_list(image=img)#, masks=mask)
            img=augmented["image"]
            # mask=augmented["masks"]
            
 
        
 
        img = img.transpose((2, 0, 1))
        
        # mask=np.array(mask)
        
        
 
        
 
        img=torch.from_numpy(img)
        # mask=torch.from_numpy(mask)
        
        img=transforms.Normalize(mean=self.means, std=self.stds)(img)
        return img, img_file[0]

        
        
dataset = BasicDataset(training_image_directory, training_mask_directory, scale=scale, transform=False)
 
#!!!!!!!!!!!!!!!!!!!!!!!!!!Set batch size here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

nimages = 0
mean = 0. 
std = 0.
for batch, _ in train_loader:
    # Rearrange batch to be the shape of [B, C, W * H]
    batch = batch.view(batch.size(0), batch.size(1), -1)
    # Update total number of images
    nimages += batch.size(0)
    # Compute mean and std here
    mean += batch.mean(2).sum(0) 
    std += batch.std(2).sum(0)
 
# Final step
mean /= nimages
std /= nimages
 
print(mean)
print(std)

m=mean.numpy()
np.save("mean_array.npy",m)

s=std.numpy()
np.save("std_array.npy",s)

print(m)
# m=np.array2string(m)

# print(m)

# m.to_csv(means)

# print(m.split("[]"))

# ''.join(filter(str.isdigit, m))

dataset.means=mean
dataset.stds=std 

nimages = 0
means = 0.
stds = 0.
for batch, _ in train_loader:
    # Rearrange batch to be the shape of [B, C, W * H]
    batch = batch.view(batch.size(0), batch.size(1), -1)
    # Update total number of images
    nimages += batch.size(0)
    # Compute mean and std here
    means += batch.mean(2).sum(0) 
    stds += batch.std(2).sum(0)
 
# Final step
means /= nimages
stds /= nimages
 
print(means)
print(stds)

tensor([0.3232, 0.3232, 0.3232], dtype=torch.float64)
tensor([0.2015, 0.2015, 0.2015], dtype=torch.float64)
[0.32321666 0.32321666 0.32321666]
tensor([-3.5774e-16, -3.5774e-16, -3.5774e-16], dtype=torch.float64)
tensor([1.0000, 1.0000, 1.0000], dtype=torch.float64)


In [ ]:
#For loop for FCN model training Cell Code Box 5
#!cd "drive/My Drive/Colab Notebooks"
# Semantic Segmentation and Data Extraction in Pytorch Using FCN by Pranav Raja
 
import torchvision
# from torchvision.models.segmentation.fcn import FCNHead
from torchvision.models.segmentation.deeplabv3 import DeepLabHead
from torch.utils.data import DataLoader, random_split
import torch
# from torch._six import container_abcs, string_classes, int_classes
import torchvision.transforms as T
import matplotlib.pyplot as plt
import torch.nn as nn
import os
import psutil
import gc
import random
 
dir_checkpoint = models_directory
 
 
model_group=model_group
num_models=num_models
os.mkdir(dir_checkpoint+model_group)
seed=0
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)
for i in range(num_models):
  #!!!!!!! Here we pull in a pretrained FCN on torch and we replace the output layer since we have six classes rather than 21!!!!!!!!
  num_classes=num_materials
  model=torchvision.models.segmentation.deeplabv3_resnet101(pretrained=False, progress=True)
  model.classifier=DeepLabHead(2048, num_classes)
  
  def trainval_split(dataset, val_fraction=0.5):
 
    validation_size = int(len(dataset) * val_fraction)
    train_size = len(dataset) - validation_size
    train, val = torch.utils.data.random_split(dataset, [train_size, validation_size], generator=torch.Generator().manual_seed(i))
 
    return train, val
 
 
 
  dataset= BasicDataset(training_image_directory, training_mask_directory, scale=scale, transform=True)
  dataset_train, dataset_val=trainval_split(dataset, val_fraction=validation_fraction)

  train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True, drop_last=True)#, collate_fn=pad_collate)
  val_loader = DataLoader(dataset_val, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)#, collate_fn=pad_collate)
 
 
  #%%
 
  # this is the train code 
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model= nn.DataParallel(model)
  model.to(device)
  #!!!!!!!!!!!!!!!!!!!!!!!!!!!! Input epochs here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
  num_epochs=num_epochs
  # read up on optimizers but Adam should work for now, if you get good results with Adam then you can try SGD (it's harder to tune but usually converges better)
  optimizer=torch.optim.Adam(model.parameters(), lr=1e-3)
 
  #just initializing a value called best_loss
  best_loss=999
 
  # choose a loss function

  criterion = nn.BCEWithLogitsLoss()

  #this is the train loop
  for epoch in range(num_epochs):
      print(psutil.virtual_memory().percent)
      print('Epoch: ', str(epoch))
    #add back if doing fractional training
      train_loader.dataset.dataset.transform=True
      model.train()
      for images, masks in train_loader:
 
          images = images.to(device=device, dtype=torch.float16) #11111111
          masks = masks.to(device=device, dtype=torch.float16)
          
          with torch.cuda.amp.autocast(): 
              #forward pass
              preds=model(images)['out'].cuda()

              #compute loss
              loss=criterion(preds, masks)
        
          #reset the optimizer gradients to 0
          optimizer.zero_grad()
 
          #backward pass (compute gradients)
          loss.backward()
 
          #use the computed gradients to update model weights
          optimizer.step()
 
          print('Train loss: '+str(loss.to('cpu').detach()))
      # model.eval()
      #add back if doing fractional training
      val_loader.dataset.dataset.transform=False
      current_loss=0
      
      #test on val set and save the best checkpoint
      model.eval()
      with torch.no_grad():
        for images, masks in val_loader:
            images = images.to(device=device, dtype=torch.float16)
            masks = masks.to(device=device, dtype=torch.float16)

            
            with torch.cuda.amp.autocast(): ##################################################################################
                preds=model(images)['out'].cuda()
                # print(preds)
                # print(masks)
                loss=criterion(preds, masks)
            #print('hi')
            current_loss+=loss.to('cpu').detach()
            del images, masks, preds, loss
  #!!!!!!!!!!!!!!!!!!!!!!!!!!!Re-name model here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!        
      if best_loss>current_loss:
          best_loss=current_loss
          print('Best Model Saved!, loss: '+ str(best_loss))
          torch.save(model.state_dict(), dir_checkpoint+model_group + current_model_name+str(i+1)+".pth")
      else:
          print('Model is bad!, Current loss: '+ str(current_loss) + ' Best loss: '+str(best_loss))
      print('\n')

11.7
Epoch:  0
Train loss: tensor(0.6011)
Train loss: tensor(0.7375)
Train loss: tensor(0.6027)
Train loss: tensor(0.5616)
Train loss: tensor(0.5446)
Train loss: tensor(0.4950)
Train loss: tensor(0.4493)
Model is bad!, Current loss: tensor(nan) Best loss: 999


11.7
Epoch:  1
Train loss: tensor(0.4408)
Train loss: tensor(0.4092)
Train loss: tensor(0.4085)
Train loss: tensor(0.4018)
Train loss: tensor(0.4086)
Train loss: tensor(0.4147)
Train loss: tensor(0.4015)
Model is bad!, Current loss: tensor(nan) Best loss: 999


11.7
Epoch:  2
Train loss: tensor(0.4042)
Train loss: tensor(0.4058)
Train loss: tensor(0.3920)
Train loss: tensor(0.3953)
Train loss: tensor(0.3973)
Train loss: tensor(0.4074)
Train loss: tensor(0.4039)
Model is bad!, Current loss: tensor(nan) Best loss: 999


11.7
Epoch:  3
Train loss: tensor(0.4061)
Train loss: tensor(0.3967)
Train loss: tensor(0.3817)
Train loss: tensor(0.3766)
Train loss: tensor(0.3993)
Train loss: tensor(0.3949)
Train loss: tensor(0.3991)
Best Model

In [35]:
#Code Box 6
import torch.nn as nn
import torchvision
import torch
from torchvision.models.segmentation.fcn import FCNHead
from torchvision.models.segmentation.deeplabv3 import DeepLabHead
from torch.utils.data import DataLoader, random_split
#from statistics import mean
import numpy as np
import pandas as pd
# interim_list=[]
modeldata = pd.DataFrame(columns=["name", "precision", "recall", "accuracy", "f1"])

 
for s in range(num_models):

  model=torchvision.models.segmentation.deeplabv3_resnet101(pretrained=True, progress=True)
  model.classifier=DeepLabHead(2048, num_materials)
 
  # model=torchvision.models.segmentation.fcn_resnet101(pretrained=False)
  #!!!!!!!!!!!!!!!!!Specify Layer # here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
  # model.classifier=FCNHead(2048, num_materials)
  device = torch.device('cuda')
 
  outputs=[]
  model.to(device)
 
  #!!!!!!!!!!!!!!!!!!!!!Select Correct Model from the best models directory!!!!!!!!!!!!!!!!!!!!!!!!!1
 
  model.load_state_dict(torch.load(models_directory+model_group + current_model_name+str(s+1)+".pth"), strict=False)
 
 
  model.eval()
 
  dataset_val = BasicDataset(test_images, test_masks, scale=scale, transform=False)
  val_loader = DataLoader(dataset_val, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)#, collate_fn=pad_collate)
 
  prop_list = []
  for mat in materials:
    prop_list.append([[],[],[],[]])
 
  for images, target in val_loader:
    images = images.to(device=device, dtype=torch.float32)
    target = target.to(device=device, dtype=torch.float32)
 
    with torch.no_grad():
      pred=model(images)['out'].cuda()
      pred=nn.Sigmoid()(pred)
    
    for i, mat in enumerate(materials):
      material_target=target[:,i,:,:]
      material_pred = pred[:, i, :, :]
      material_pred[material_pred >=mat.confidence_threshold] = 1
      material_pred[material_pred <=mat.confidence_threshold] = 0
      pred[:, i, :, :]=material_pred
 
      material_tp=torch.sum(material_target*material_pred, (1,2))
      material_fp=torch.sum((1-material_target)*material_pred, (1,2))
      material_fn=torch.sum(material_target*(1-material_pred), (1,2))
      material_tn=torch.sum((1-material_target)*(1-material_pred), (1,2))
 
     
 
      material_precision=torch.mean((material_tp+0.000000001)/(material_tp+0.000000001+material_fp))
      material_recall=torch.mean((material_tp+0.000000001)/(material_tp+0.000000001+material_fn))
      material_accuracy=torch.mean((material_tp+material_tn)/(material_tp+material_tn+material_fp+material_fn))
      material_f1=torch.mean((material_tp+0.000000001)/((material_tp+0.000000001)+0.5*(material_fp+material_fn)))
 
    
      prop_list[i][0].append(material_precision.cpu().detach().numpy())
      prop_list[i][1].append(material_recall.cpu().detach().numpy())
      prop_list[i][2].append(material_accuracy.cpu().detach().numpy())
      prop_list[i][3].append(material_f1.cpu().detach().numpy())
      
 
          
 
 
 
 
  model_name=current_model_name
  model_number=(str(s+1))
  print(model_name)
 
  #printing with pandas
  properties = {"name" : [mat.name for mat in materials],
                "precision" : [str(np.mean(prop_list[i][0])) for i in range(num_materials)],
                "recall" : [str(np.mean(prop_list[i][1])) for i in range(num_materials)],
                "accuracy" : [str(np.mean(prop_list[i][2])) for i in range(num_materials)],
                "f1" : [str(np.mean(prop_list[i][3])) for i in range(num_materials)]}
  df = pd.DataFrame(properties, columns = ["name", "precision", "recall", "accuracy", "f1"])
  df=pd.concat([df, pd.DataFrame(columns=["model number","model name"])])
  df[["model number","model name"]]=[model_number, model_name]
  # display(df)
  
  modeldata=modeldata.append([df], ignore_index=True)


display(modeldata)



amf p2 1p 100 epoch


/tmp/ipykernel_37041/1984061536.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  modeldata=modeldata.append([df], ignore_index=True)


,name,precision,recall,accuracy,f1,model number,model name
0,amf,1.0,0.041666668,0.9997401,0.041666668,1,amf p2 1p 100 epoch
1,background,0.9581894,1.0,0.9581894,0.95839864,1,amf p2 1p 100 epoch


In [17]:
#Code Box 7
display(modeldata)
modeldata.to_csv(csv_directory)

,name,precision,recall,accuracy,f1,model number,model name
0,amf,0.44447887,0.3880566,0.7211492,0.11118,1,dl 5 materials soil bce p2 1p 100 epoch
1,background,0.9440948,1.0,0.9440948,0.9443667,1,dl 5 materials soil bce p2 1p 100 epoch


In [14]:
#Code Box 8
"""Input model number here"""
model_number='1'

In [15]:
#Code Box 9
import torch
import torch.nn as nn
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import torchvision
import torchvision.transforms as T
import numpy as np
from torchvision.models.segmentation.fcn import FCNHead
from torchvision.models.segmentation.deeplabv3 import DeepLabHead
 
 
import time





 
model=torchvision.models.segmentation.deeplabv3_resnet101(pretrained=True, progress=True)
model.classifier=DeepLabHead(2048, num_materials)
 
device = torch.device('cuda')
 
outputs=[]
model.to(device)
 
#!!!!!!!!!!!!!!!!!!!!!Select Correct Model from the best models directory!!!!!!!!!!!!!!!!!!!!!!!!!1
model.load_state_dict(torch.load(models_directory+model_group + current_model_name+model_number+'.pth'), strict=False)
 
 
 
 
model.eval()

DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [16]:

import os
import pandas as pd
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from glob import glob
import argparse
import torch
import torchvision
from PIL import Image
import torch.nn as nn
import scipy as scipy

from scipy import ndimage as ndi

from skimage import (
    color, feature, filters, measure, morphology, segmentation, util)

from glob import glob
import torchvision.transforms as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from scipy.ndimage import morphology
sys.path.append(os.path.join(sys.path[0]))  # To find local version of the library
from skimage.filters import sobel
from scipy import ndimage
from torchvision.models.detection.rpn import AnchorGenerator
import torch
from skimage.color import rgb2gray, label2rgb
from skimage.segmentation import slic, join_segmentations, watershed
from skimage import morphology
from skimage.morphology import selem


from skimage import io,exposure, feature, filters, io, measure, morphology, restoration, segmentation, transform, util, data, color
from skimage.measure import label, regionprops
from skimage.transform import rescale, resize, downscale_local_mean
import os
import numpy as np
input_path = inference_directory
filenames = os.listdir(input_path)

output_path=output_directory

dataset_val = InferenceDataset(input_path, 1, transform=False)

# dataset_val.means=means
# dataset_val.stds=stds

val_loader = DataLoader(dataset_val, batch_size=1,shuffle=False, num_workers=0, pin_memory=True)#, collate_fn=pad_collate)
model.eval()


whole_leaf_tables=[]
file_name=[]
color=[]
value_counts=[]

for images, filename in val_loader:
    filename=filename[0]
    new_dir_name = output_path
    if not os.path.exists(new_dir_name):
      os.makedirs(new_dir_name)
    
    for mat in materials:
      new_dir_name_mat= new_dir_name + mat.name
      if not os.path.exists(new_dir_name_mat):
        os.makedirs(new_dir_name_mat)
    print(images.shape)
    print(filename)
    images = images.to(device=device, dtype=torch.float32)
    # masks = masks.to(device=device, dtype=torch.float32)
    with torch.no_grad():
      preds=model(images)['out'].cuda()
      preds=nn.Sigmoid()(preds)
      mat_mask1=preds.cpu().detach().numpy()[0,0,:,:]
      mat_mask1[mat_mask1 >= mat.confidence_threshold] = mat.output_val
      mat_mask1[mat_mask1 < mat.confidence_threshold] = 0  
      list_of_mat_tables = []
      for i, mat in enumerate(materials):

        mat_mask = preds.cpu().detach().numpy()[0,i,:,:]
        # plt.imshow(mat_mask)
        # plt.show()

        mat_mask[mat_mask >= mat.confidence_threshold] = mat.output_val
        mat_mask[mat_mask < mat.confidence_threshold] = 0
        # plt.imshow(mat_mask)
        # plt.show()

        # distance = ndi.distance_transform_edt(mat_mask)

        # local_max_coords = feature.peak_local_max(distance, min_distance=30)
        # local_max_mask = np.zeros(distance.shape, dtype=bool)
        # local_max_mask[tuple(local_max_coords.T)] = True
        # markers = measure.label(local_max_mask)

        # seg1 = segmentation.watershed(-distance, markers, mask=mat_mask, compactness=.5)
        # seg1=np.array(seg1, dtype=float)
        
        # sobel = scipy.ndimage.sobel(mat_mask)
        # # io.imshow(sobel)
        # markers = np.zeros_like(sobel)
        # foreground, background = 1, 2
        # markers[sobel <= 0] = background
        # markers[sobel >=1] = foreground
        

        # ws = watershed(sobel,mask=mat_mask, compactness=1)
        
        # seg1 = label(ws == foreground)
        # selem=morphology.selem.disk(8)
        # mat_mask=morphology.binary_erosion(mat_mask, selem=selem)


        img=images.cpu().detach().numpy()[0,:,:,:]
        img=np.transpose(img,(1,2,0))
        img=rgb2gray(img)
        img2=label(mat_mask, background=0)
        img3=label2rgb(img2, img, alpha=0.3, bg_label=0)   
   
        # io.imsave(output_path+filename.split('.png')[0]+'_' + mat.name + "_mask.png", mat_mask)
        # io.imsave(new_dir_name+"/"+ mat.name + "/"+filename.split(".")[0].split("/")[0]+'_' + mat.name + "_mask.png", mat_mask)
        # io.imsave(new_dir_name+"/"+filename.split(".")[0].split('/')[-1]+"_mask.png", img3)
        # io.imsave(new_dir_name+'/'+filename.split(following)[0]+'.png', img3) 

        io.imsave(new_dir_name+'/' + mat.name + '/'+filename.split(following)[0].split(proceeding)[-1]+'_' + mat.name + "_mask.png", mat_mask)
        io.imsave(new_dir_name+'/'+filename.split(following)[0].split(proceeding)[-1]+'_'+"_mask.png", img3) 
       
        np_mat = np.array(mat_mask)
        
        
        label_mat = label(np_mat, background = 0)
        
        if np.sum(label_mat)<1:
          label_mat=np.ones_like(label_mat)
        elif np.sum(label_mat)>=1:
          label_mat=label_mat
       
        
        mat_table=measure.regionprops_table(label_mat, img, properties=['label','area','perimeter'])
        print(len(mat_table['label']))
        count=(len(mat_table['label']))
        mat_table=pd.DataFrame(mat_table)
        if mat_table['area'].sum()==np.ones_like(label_mat).sum():
          mat_table_a=mat_table['area'].sum()==0
        
        elif mat_table['area'].sum()<np.ones_like(label_mat).sum():
          mat_table_a=mat_table['area'].sum()

        if mat_table['area'].sum()==np.ones_like(label_mat).sum():
          mat_table_p=mat_table['perimeter'].sum()==0
      
        elif mat_table['area'].sum()<np.ones_like(label_mat).sum():
          mat_table_p=mat_table['perimeter'].sum()



        mat_table_c=count
        # mat_table_p=mat_table['perimeter'].sum()

        list_of_mat_tables.append(mat_table_a)
        list_of_mat_tables.append(mat_table_p)
        list_of_mat_tables.append(mat_table_c)
        


      # io.imshow(mat_mask[:,:])
      print(list_of_mat_tables)
      list_of_mat_tables = np.array(list_of_mat_tables, dtype=int)
      whole_leaf = value_counts.append(list_of_mat_tables)
      print(whole_leaf)

      name = file_name.append([filename])
      
  
counts=(np.array(value_counts))
print(counts)
names=(np.array(file_name))
print(names)

whole_leaf=np.concatenate((names, counts), axis=1)

whole_leaf_table=(pd.DataFrame(whole_leaf))
print(whole_leaf_table)
table_columns = [[f"{mat.name} area(pix)", f"{mat.name} perimeter (pix)", f"{mat.name} counts"] for mat in materials]
print(table_columns)
table_columns = [element for sublist in table_columns for element in sublist] #flattening the list
print(table_columns)
table_columns = ["file_name"] + table_columns
print(table_columns)
whole_leaf_table.columns = table_columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row',None)
# print(d)
#!!!!!!!!!!!!!!!!!!!!!!!!!!Can change table output name here!!!!!!!!!!!!!!!!!!!!!!!!!!
whole_leaf_table.to_csv(new_dir_name+'/'+'material area, perimeter, and counts.csv')
#print (counts)
print('----end-----')
      


    




torch.Size([1, 3, 1920, 2000])
/pscratch/sd/d/drippner/Reslice Images/Reslice of T4PmR2d1h11922.png


/tmp/ipykernel_37041/10124390.py:133: UserWarning: /pscratch/sd/d/drippner/amf test//amf/1922_amf_mask.png is a low contrast image
  io.imsave(new_dir_name+'/' + mat.name + '/'+filename.split(following)[0].split(proceeding)[-1]+'_' + mat.name + "_mask.png", mat_mask)


1


/tmp/ipykernel_37041/10124390.py:133: UserWarning: /pscratch/sd/d/drippner/amf test//background/1922_background_mask.png is a low contrast image
  io.imsave(new_dir_name+'/' + mat.name + '/'+filename.split(following)[0].split(proceeding)[-1]+'_' + mat.name + "_mask.png", mat_mask)


1
[False, False, 1, False, False, 1]
None
torch.Size([1, 3, 1920, 2000])
/pscratch/sd/d/drippner/Reslice Images/Reslice of T4PmR2d1h10282.png


...

1


/tmp/ipykernel_37041/10124390.py:133: UserWarning: /pscratch/sd/d/drippner/amf test//background/1892_background_mask.png is a low contrast image
  io.imsave(new_dir_name+'/' + mat.name + '/'+filename.split(following)[0].split(proceeding)[-1]+'_' + mat.name + "_mask.png", mat_mask)


1
[False, False, 1, False, False, 1]
None
torch.Size([1, 3, 1920, 2000])
/pscratch/sd/d/drippner/Reslice Images/Reslice of T4PmR2d1h10658.png


/tmp/ipykernel_37041/10124390.py:133: UserWarning: /pscratch/sd/d/drippner/amf test//amf/0658_amf_mask.png is a low contrast image
  io.imsave(new_dir_name+'/' + mat.name + '/'+filename.split(following)[0].split(proceeding)[-1]+'_' + mat.name + "_mask.png", mat_mask)


KeyboardInterrupt: 